In [1]:
import random
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import cloudpickle as cpkl
from pathlib import Path
import numpy as np
import datetime
from datetime import timedelta
from functools import reduce
from fbprophet import Prophet
import multiprocessing as mp
from itertools import repeat
from joblib import Parallel, delayed
import calendar

In [2]:
script_name='prophet_item_store'
train_pkl_file = 'input/train_pkl'
test_pkl_file = 'input/test_pkl'
train,transactions,stores,oil,items,holidays_events = cpkl.loads(open(train_pkl_file,'rb').read())

In [3]:
big_bang = datetime.date(2014,12,31)
maxdate = datetime.date(2017,8,31)
train = train.loc[train.date>big_bang]
#train.loc[:, "onpromotion"].fillna(False, inplace=True)
transactions = transactions.loc[transactions.date>big_bang]

In [4]:
holidays_events.date = pd.to_datetime(holidays_events.date)
holidays_events.date = holidays_events.date.dt.date
all_days = pd.DataFrame({'date':pd.date_range(big_bang,maxdate,freq="D")})
all_days.date = all_days.date.dt.date
all_days['holiday']=0
all_days.loc[all_days.date.isin(holidays_events.date),'holiday']=1
train.date = train.date.dt.date
transactions.date = transactions.date.dt.date
oil.date = oil.date.dt.date

In [5]:
holidays_events = holidays_events[holidays_events.transferred==False]
oil.dcoilwtico.fillna(method='ffill',inplace=True)
print(oil.head())
import gc
gc.collect()

         date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20


47

In [6]:
class elapsed(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        self.last_store = 0
        self.last_item = 0
        
    def get(self, row):
        if (row.store_nbr != self.last_store) or (row.item_nbr != self.last_item):
            self.last = pd.to_datetime(np.nan)
            self.last_store = row.store_nbr
            self.last_item = row.item_nbr
        if (row[self.fld]): self.last = row.date
        return (row.date-self.last).days

class elapsed_date(object):
    def __init__(self, fld):
        self.fld = fld
        self.last = pd.to_datetime(np.nan)
        
    def get(self, row):
        if (row[self.fld]): self.last = row.date
        return (row.date-self.last).days
gc.collect()


60

In [7]:
def add_elapsed_date(fld, prefix):
    sh_el = elapsed_date(fld)
    all_days[prefix+fld] = all_days.apply(sh_el.get, axis=1)
fld='holiday'
all_days = all_days.sort_values(['date'])
add_elapsed_date(fld, 'After')
all_days = all_days.sort_values([ 'date'], ascending=[False])
add_elapsed_date(fld, 'Before')
all_days.loc[all_days.date==datetime.date(2017,8,25),'Beforeholiday']=-34
all_days.loc[all_days.date==datetime.date(2017,8,26),'Beforeholiday']=-33
all_days.loc[all_days.date==datetime.date(2017,8,27),'Beforeholiday']=-32
all_days.loc[all_days.date==datetime.date(2017,8,28),'Beforeholiday']=-31
all_days.loc[all_days.date==datetime.date(2017,8,29),'Beforeholiday']=-30
all_days.loc[all_days.date==datetime.date(2017,8,30),'Beforeholiday']=-29
all_days.loc[all_days.date==datetime.date(2017,8,31),'Beforeholiday']=-28

In [8]:
all_days.set_index('date',inplace=True)
bwd_days = all_days[['holiday']].sort_index().rolling(7,min_periods=1).sum().reset_index()
fwd_days = all_days[['holiday']].sort_index(ascending=False).rolling(7,min_periods=1).sum().reset_index()
all_days.reset_index(inplace=True)

In [9]:
all_days = all_days.merge(bwd_days,on='date',how="left",suffixes=['','_bw'],copy=False)
del bwd_days
all_days = all_days.merge(fwd_days,on='date',how="left",suffixes=['','_fw'],copy=False)
del fwd_days
all_days

,date,holiday,Afterholiday,Beforeholiday,holiday_bw,holiday_fw
0,2017-08-31,0,7,-28.0,0.0,0.0
1,2017-08-30,0,6,-29.0,1.0,0.0
2,2017-08-29,0,5,-30.0,1.0,0.0
3,2017-08-28,0,4,-31.0,1.0,0.0
4,2017-08-27,0,3,-32.0,1.0,0.0
5,2017-08-26,0,2,-33.0,1.0,0.0
6,2017-08-25,0,1,-34.0,1.0,0.0
7,2017-08-24,1,0,0.0,1.0,1.0
8,2017-08-23,0,8,-1.0,0.0,1.0
9,2017-08-22,0,7,-2.0,0.0,1.0


In [10]:
def is_salary_day(date):
    return 1 if (date.day == 15 or calendar.monthrange(date.year,date.month)[1]==date.day) else 0
train['salary_day'] = train.date.apply(is_salary_day)

In [11]:
all_days = all_days[all_days.date.isin(train.date.unique())]
import pickle
pickle.dump(all_days,open("results/all_days.pkl",'wb'))

In [12]:
all_days.shape

(956, 6)

In [13]:
print(all_days.head())
print(train.shape)

          date  holiday  Afterholiday  Beforeholiday  holiday_bw  holiday_fw
16  2017-08-15        1             0            0.0         3.0         1.0
17  2017-08-14        0             3           -1.0         2.0         1.0
18  2017-08-13        0             2           -2.0         2.0         1.0
19  2017-08-12        0             1           -3.0         2.0         1.0
20  2017-08-11        1             0            0.0         3.0         2.0
(86902776, 7)


In [14]:
joined = train.merge(all_days,on='date',how='left',copy=False)
del train
import gc
gc.collect()

203

In [15]:
print(joined.shape)
df = pickle.load(open('results/onpromotion_features.pickle','rb'))
print(df.shape)

(86902776, 12)
(86902776, 8)


In [16]:
df.head()

,date,store_nbr,item_nbr,onpromotion,Afteronpromotion,Beforeonpromotion,onpromotion_bw,onpromotion_fw
0,2017-08-10,1,96995,0,0.0,0.0,0.0,0.0
1,2017-08-07,1,96995,0,0.0,0.0,0.0,0.0
2,2017-08-06,1,96995,0,0.0,0.0,0.0,0.0
3,2017-08-03,1,96995,0,0.0,0.0,0.0,0.0
4,2017-08-02,1,96995,0,0.0,0.0,0.0,0.0


In [17]:
joined = joined.merge(df,on=['store_nbr','item_nbr','date'],copy=False)
print(joined.shape)

(86902776, 17)


In [18]:
stores.columns

Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')

In [19]:
del df
gc.collect()
joined = joined.merge(stores,on='store_nbr',how="left",copy=False)
print(joined.shape)

(86902776, 21)


In [20]:
joined = joined.merge(items,on='item_nbr',how="left",copy=False)
print(joined.shape)

(86902776, 24)


In [21]:
print(holidays_events.columns)
print(type(holidays_events.date[0]))
holidays_events = holidays_events[['date','type','locale','locale_name','description']]

Index(['date', 'type', 'locale', 'locale_name', 'description', 'transferred'], dtype='object')
<class 'datetime.date'>


In [22]:
print(len(holidays_events.date.unique()))
print(holidays_events.shape)

300
(338, 5)


In [23]:
joined = joined.merge(holidays_events,on='date',how="left",copy=False)
print(joined.shape)

(88776319, 28)


In [24]:
joined = joined.merge(oil,on="date",how="left",copy=False)
print(joined.shape)

(88776319, 29)


In [25]:
gc.collect()
joined = joined.merge(transactions,on=['date','store_nbr'],how="left",copy=False)
print(joined.shape)
print(joined.head())
gc.collect()

(88776319, 30)
         id        date  store_nbr  item_nbr  unit_sales  onpromotion_x  \
0  38594264  2015-01-01         25    103665        12.0          False   
1  38594265  2015-01-01         25    105575        23.0          False   
2  38594266  2015-01-01         25    108634         1.0          False   
3  38594267  2015-01-01         25    108698         6.0          False   
4  38594268  2015-01-01         25    108786         6.0          False   

   salary_day  holiday  Afterholiday  Beforeholiday     ...       cluster  \
0           0        1             0            0.0     ...             1   
1           0        1             0            0.0     ...             1   
2           0        1             0            0.0     ...             1   
3           0        1             0            0.0     ...             1   
4           0        1             0            0.0     ...             1   

         family  class  perishable   type_y    locale  locale_name  \
0

297

In [26]:
#pickle.dump(joined,open('results/joined.pickle','wb'))
gc.collect()
def add_datepart(df,column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df["Year"] = df[column_name].dt.year
    df["Month"] = df[column_name].dt.month
    df["Week"] = df[column_name].dt.week
    df["Day"] = df[column_name].dt.day
add_datepart(joined,'date')

In [27]:
joined.to_csv('results/joined.csv')

In [28]:
gc.collect()
(joined.date[0].year)

2015

In [29]:
joined.columns

Index(['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion_x',
       'salary_day', 'holiday', 'Afterholiday', 'Beforeholiday', 'holiday_bw',
       'holiday_fw', 'onpromotion_y', 'Afteronpromotion', 'Beforeonpromotion',
       'onpromotion_bw', 'onpromotion_fw', 'city', 'state', 'type_x',
       'cluster', 'family', 'class', 'perishable', 'type_y', 'locale',
       'locale_name', 'description', 'dcoilwtico', 'transactions', 'Year',
       'Month', 'Week', 'Day'],
      dtype='object')

In [32]:
(joined.Beforeholiday.unique())

array([  0.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1., -36., -35., -34.,
       -33., -32., -31., -30., -29., -28., -27., -26., -25., -24., -23.,
       -22., -21., -20., -19., -18., -17., -16., -15., -14., -13., -12.,
       -11., -10.,  -9.,  -8., -37., -55., -54., -53., -52., -51., -50.,
       -49., -48., -47., -46., -45., -44., -43., -42., -41., -40., -39.,
       -38.])